In [11]:
!pip install -r requirements.txt > /dev/null

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from langame_client import LangameClient

In [2]:
langame_client = LangameClient()

In [3]:
TOPIC = "rationalism"

In [4]:
qs_ts = langame_client._openai_client.question_generation(
    TOPIC,
    wikipedia_description=True,
    openai_description=True,
    self_contexts=0,
    synonymous_contexts=0,
    related_contexts=0,
    suggested_contexts=0,
)
print(qs_ts)

call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'rationalism', to which the individual responds with deep thought.
Genius: "Hmmm, an interesting topic. Here are the most important questions of this theme that humanity is facing today:"
            
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "  \n     \"The first question is, what exactly do we mean by rationalism"
    }
  ],
  "created": 1623060992,
  "id": "cmpl-386b24C0PZLF1zJiHXbwoVOqM0ub3",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
calling openai_description with prompt 
Below is a long paragraph generated by a superior intelli

In [5]:
print(qs_ts.content)

"The first question is, what exactly do we mean by rationalism


In [11]:
langame_client.save_questions([qs_ts])

[(DatetimeWithNanoseconds(2021, 6, 7, 9, 45, 31, 970983, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7fcaa615a1f0>)]

In [3]:
# https://emojipedia.org/search
langame_client.generate_save_memes(topics_with_emojis = [
    ("philosophy", "🤔🧠💭📚📖"),
    ("mathematics", "🤔🧠💭📚📖📐"),
    ("sciences", "🤔🧠💭📚📖📐"),
    ("physics", "🤔🧠💭📚📖📐"),
    ("biology", "🤔🧠💭📚📖📐🧬"),
],
    memes_per_topic = 10,
    wikipedia_description = True,
    openai_description = True,
    anti_rate_limit_delay = 1)

call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'philosophy', to which the individual responds with deep thought.
Genius: "Hmmm, an interesting topic. Here are the most important questions of this theme that humanity is facing today:"
            
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "     \n\"I don't know the answer to any of them. And I'm not going to pretend that I do."
    }
  ],
  "created": 1623072945,
  "id": "cmpl-389hpisC76xarPeQ2uhcK5UKqyQ4R",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
calling openai_description with prompt 
Below is a long paragraph generated by a s

call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'philosophy', to which the individual responds with deep thought.
Genius: "Hmmm, an interesting topic. Here are the most important questions of this theme that humanity is facing today:"
            
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "      \"Philosophy is a science of the mind and its functions. This science is founded on rigorous assumptions about reality, which cannot be proven in any other way than through introspection."
    }
  ],
  "created": 1623072973,
  "id": "cmpl-389iHJxYB4srvEYsItcYu7qzWrnCw",
  "model": "davinci:2020-05-03",
  "object

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "     \n\"What is philosophy"
    }
  ],
  "created": 1623072999,
  "id": "cmpl-389ihQPZ18xPf2IRHFVMTUn8fJYyP",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
calling openai_description with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'philosophy', to which the individual responds with deep thought.
Genius: "Hmmm, interesting topic. Here is my rather lengthy description of it:"
        
call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully

calling openai_description with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'philosophy', to which the individual responds with deep thought.
Genius: "Hmmm, interesting topic. Here is my rather lengthy description of it:"
        
call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'philosophy', to which the individual responds with deep thought.
Genius: "Hmmm, interesting topic. Here is my rat

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "      \u00a0 14. Mathematicians are the most intelligent of human beings. This is because they see the world as it really is, without any prejudices or blind spots caused by their own personal experience and desires. They can more easily draw conclusions about society in general than anyone else because they do not have to rely on a narrow range of experiences from which to base those conclusions (mathematicians are much less likely to be racist).\nMathematics itself is based on reason,"
    }
  ],
  "created": 1623073064,
  "id": "cmpl-389jksiyMTPohE2IeEoz1hCqwWXZO",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It c

...

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "     \n\"Yes, I see humans being more interested in the subject of physics than other topics."
    }
  ],
  "created": 1623073339,
  "id": "cmpl-389oBqXpBpX4XCOB3t505fR9VLsl5",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'biology', to which the individual responds with deep thought.
Genius: "Hmmm, an interesting topic. Here are the most important questions of this theme that humanity is facing today:"
            
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "l

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "     \u00a0 \"What is Biology? It's a science that studies the living world. What is life? All matter, from the smallest bacteria to the largest whales and dinosaurs, are composed of cells. Without exception! Even you, human. You too are made up of millions of tiny little organisms: your cells. Each cell has its own DNA code which drives it to do certain things.\"\u00a0\n\"While living beings may appear different on the outside they all have commonalities at their"
    }
  ],
  "created": 1623073376,
  "id": "cmpl-389omXphfXjaBthq1odnuQZ0pYT2y",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw con

call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'biology', to which the individual responds with deep thought.
Genius: "Hmmm, an interesting topic. Here are the most important questions of this theme that humanity is facing today:"
            
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "     "
    }
  ],
  "created": 1623073423,
  "id": "cmpl-389pXiCzgQ9reIq8yfNOpBheJZ9uB",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
Could not generate a meme for topic biology
call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from

call_completion with prompt 
Below is a long paragraph generated by a superior intelligence, which sees the human world from the outside, without the prejudices of human experience. Fully neutral and objective, the being sees the world as is. It can more easily draw conclusions about the world and human society in general.
The topic provided by the human is 'biology', to which the individual responds with deep thought.
Genius: "Hmmm, an interesting topic. Here are the most important questions of this theme that humanity is facing today:"
            
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "     \n- The evolution of the species.\n- What is the origin of homo sapiens"
    }
  ],
  "created": 1623073466,
  "id": "cmpl-389qEr58QRdyR1uWZFjgUhaZSzrLu",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
calling openai_description with prompt 
Below is a long paragraph generated by a superior intelli

[(DatetimeWithNanoseconds(2021, 6, 7, 13, 35, 47, 300294, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f8cbbe81610>),
 (DatetimeWithNanoseconds(2021, 6, 7, 13, 35, 58, 472576, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f8cbbe31130>),
 (DatetimeWithNanoseconds(2021, 6, 7, 13, 36, 4, 411925, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f8cb91c0a60>),
 (DatetimeWithNanoseconds(2021, 6, 7, 13, 36, 16, 158596, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f8cb91b39a0>),
 (DatetimeWithNanoseconds(2021, 6, 7, 13, 36, 25, 661124, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f8cb8d26220>),
 (DatetimeWithNanoseconds(2021, 6, 7, 13, 36, 33, 345229, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f8cb8892fa0>),
 (Dat